In [11]:
import pandas as pd
import numpy as np
import googlemaps
import folium
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances_argmin_min

class CustomKMeans(KMeans):
    def __init__(self, n_clusters, existing_locations, penalty_factor=1000, *args, **kwargs):
        super().__init__(n_clusters, *args, **kwargs)
        self.existing_locations = existing_locations
        self.penalty_factor = penalty_factor

    def _transform_custom(self, X):
        distances_to_existing = pairwise_distances_argmin_min(X, self.existing_locations)[1]
        penalties = self.penalty_factor / distances_to_existing
        distances = super()._transform(X)
        return distances + penalties[:, np.newaxis]

    def _transform(self, X):
        return self._transform_custom(X)

    def fit(self, X, y=None, sample_weight=None):
        return super().fit(X, y, sample_weight)

# Load and preprocess data
data = pd.read_csv('stations_filtered.csv')

# Initialize the Google Maps API client
API_KEY = 'use_your_own_api_key'
gmaps = googlemaps.Client(key=API_KEY)

# Geocode addresses
def geocode_address(address):
    location = gmaps.geocode(address)
    if location:
        return location[0]['geometry']['location']
    return None

data['location'] = data['Address'].apply(geocode_address)
data = data.dropna(subset=['location'])

# Filter data to keep only charging stations within Boulder
boulder_coords = (40.014985, -105.270546)
radius = 30  # Radius in km to filter around Boulder

def is_within_radius(location, center, radius):
    R = 6371  # Earth's radius in km
    lat1, lon1 = np.radians(location)
    lat2, lon2 = np.radians(center)

    dlat = lat2 - lat1
    dlon = lon2 - lon1

    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))

    distance = R * c
    return distance <= radius

data = data[data['location'].apply(lambda x: is_within_radius((x['lat'], x['lng']), boulder_coords, radius))]

# Set problem parameters
P = 5  # Number of new charging stations
demand_increase_factor = 1.5  # Example: 1.5 represents a 50% increase in demand

# Prepare the data for clustering
weighted_points = []
for k in data.index:
    for _ in range(int(data.loc[k, 'Energy__kWh_'] * demand_increase_factor)):
        weighted_points.append((data.loc[k, 'location']['lat'], data.loc[k, 'location']['lng']))

# Initialize k-means clustering with random centers within Boulder
init_centers = np.random.uniform(low=(boulder_coords[0] - 0.05, boulder_coords[1] - 0.05),
                                  high=(boulder_coords[0] + 0.05, boulder_coords[1] + 0.05),
                                  size=(P, 2))

existing_locations = np.array(list(data['location'].apply(lambda x: [x['lat'], x['lng']])))
# Run the custom k-means clustering algorithm
kmeans = CustomKMeans(n_clusters=P, random_state=0, init=init_centers, n_init=1, existing_locations=existing_locations).fit(weighted_points) 

#Create a map centered on Boulder, Colorado
map_ = folium.Map(location=boulder_coords, zoom_start=12)

#Add existing charging station locations to the map
for index, row in data.iterrows():
    folium.Marker(location=[row['location']['lat'], row['location']['lng']],popup=row['Station_Name'],icon=folium.Icon(color='blue', icon='plug', prefix='fa')).add_to(map_)
    
#Add new charging station locations to the map
for i, center in enumerate(kmeans.cluster_centers_):
    folium.Marker(location=center,popup=f'New Charging Station {i + 1}',icon=folium.Icon(color='green', icon='plug', prefix='fa')).add_to(map_)
    
#Save the map to an HTML file
map_.save('charging_stations_map.html')

#Print the new charging station coordinates
for i, center in enumerate(kmeans.cluster_centers_):
    print(f"New charging station {i + 1} at: {center}")

New charging station 1 at: [  40.02538984 -105.24804409]
New charging station 2 at: [  40.023776 -105.180846]
New charging station 3 at: [  40.017915 -105.279955]
New charging station 4 at: [  39.9746374 -105.2486267]
New charging station 5 at: [  40.0184704 -105.2752993]
